In [1]:
using Flux
using Flux: Data.DataLoader
using Flux: onehotbatch, onecold, crossentropy, throttle
using Flux: @epochs
using Statistics
using MLDatasets
using Serialization: serialize, deserialize
using PyCall
using ImageView
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON, LinearAlgebra

In [2]:
x_train, y_train = SVHN2.traindata()
x_test, y_test = SVHN2.testdata()

(FixedPointNumbers.N0f8[0.149N0f8 0.153N0f8 … 0.196N0f8 0.188N0f8; 0.153N0f8 0.153N0f8 … 0.2N0f8 0.188N0f8; … ; 0.165N0f8 0.169N0f8 … 0.176N0f8 0.173N0f8; 0.153N0f8 0.153N0f8 … 0.165N0f8 0.165N0f8]

FixedPointNumbers.N0f8[0.404N0f8 0.408N0f8 … 0.459N0f8 0.451N0f8; 0.408N0f8 0.408N0f8 … 0.463N0f8 0.451N0f8; … ; 0.404N0f8 0.396N0f8 … 0.455N0f8 0.451N0f8; 0.38N0f8 0.38N0f8 … 0.443N0f8 0.443N0f8]

FixedPointNumbers.N0f8[0.235N0f8 0.239N0f8 … 0.298N0f8 0.29N0f8; 0.239N0f8 0.239N0f8 … 0.302N0f8 0.29N0f8; … ; 0.243N0f8 0.247N0f8 … 0.282N0f8 0.278N0f8; 0.224N0f8 0.224N0f8 … 0.271N0f8 0.278N0f8]

FixedPointNumbers.N0f8[0.506N0f8 0.525N0f8 … 0.541N0f8 0.514N0f8; 0.498N0f8 0.522N0f8 … 0.51N0f8 0.478N0f8; … ; 0.482N0f8 0.494N0f8 … 0.396N0f8 0.435N0f8; 0.482N0f8 0.49N0f8 … 0.439N0f8 0.482N0f8]

FixedPointNumbers.N0f8[0.557N0f8 0.588N0f8 … 0.596N0f8 0.569N0f8; 0.561N0f8 0.584N0f8 … 0.565N0f8 0.533N0f8; … ; 0.525N0f8 0.537N0f8 … 0.42N0f8 0.463N0f8; 0.529N0f8 0.537N0f8 … 0.463N0f8 0.51N0f8]

FixedPoin

In [3]:
epochs = 100;
batch_size = 64;

In [4]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[:,:,1, 1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] .= Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 1:10)
    return (X_batch, Y_batch)
end

make_minibatch (generic function with 1 method)

In [5]:
mb_idxs = partition(1:length(x_train[:,:,:,1]), batch_size)
train_set = [make_minibatch(x_train, y_train, i) for i in mb_idxs];
mb_idxs_test = partition(1:length(x_test[:,:,:,1]), batch_size)
x_test = x_test[:,:,:,3072] #zmniejszam próbe testową bo mam za mało pamięci
test_set = make_minibatch(x_test, y_test, 1:length(x_test));

In [6]:
function loss(x, y)
    x_aug = x .+ 0.1f0*randn(eltype(x), size(x)) #szum losowy
    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
opt = ADAM(0.001)

ADAM(0.001, (0.9, 0.999), IdDict{Any, Any}())

In [51]:
model = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    #Dense(32, 10, relu),
    softmax,
);

In [52]:
Flux.train!(loss, params(model), train_set, opt)

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 10 and 32")